# **Clasificación Automatizada de Reseñas Clínicas con Modelos Transformer**

### **Introducción y Objetivo**

Este notebook detalla la implementación de un pipeline de Procesamiento de Lenguaje Natural (NLP) para la clasificación de sentimientos en reseñas clínicas. El objetivo es doble: primero, aplicar un modelo Transformer de vanguardia, específicamente `BERT multilingüe`, para una tarea de clasificación en español; y segundo, realizar un análisis crítico de su rendimiento, limitaciones y ventajas en comparación con métodos clásicos.

Este enfoque no solo demuestra una aplicación práctica del NLP moderno, sino que también profundiza en la lógica y la teoría que hacen a los modelos Transformer tan efectivos.

-----

## **Sección 1: Preparación del Entorno**

Antes de ejecutar cualquier análisis, es fundamental establecer un entorno de trabajo robusto y bien configurado. Este primer bloque de código se encarga de varias tareas críticas:

1.  **Instalación de Dependencias**: La línea comentada `# !pip install ...` es una salvaguarda para asegurar que las tres librerías clave (`transformers`, `torch`, `pandas`) estén disponibles. `transformers` de Hugging Face es el ecosistema que nos da acceso al modelo; `torch` es el framework de deep learning subyacente que ejecuta los cálculos tensoriales; y `pandas` es la herramienta estándar para la manipulación de datos estructurados.
2.  **Importación de Módulos**: Importamos los componentes específicos que usaremos. `pipeline` es una abstracción de alto nivel de Hugging Face que simplifica enormemente el proceso de inferencia, encapsulando la tokenización, la alimentación del modelo y el post-procesamiento de los resultados.
3.  **Verificación del Entorno**: Se realizan comprobaciones para informar al usuario sobre la configuración actual. Verificar la versión de `PyTorch` es útil para la depuración y reproducibilidad. Más importante aún, se detecta la disponibilidad de una **GPU (Unidad de Procesamiento Gráfico)**. Los modelos Transformer son computacionalmente intensivos, y ejecutar la inferencia en una GPU puede ser órdenes de magnitud más rápido que en una CPU. Esta verificación nos permitirá más adelante asignar el modelo al hardware óptimo de forma dinámica.

<!-- end list -->

In [ ]:
# ===================================================================
# SECCIÓN 1: PREPARACIÓN DEL ENTORNO
# ===================================================================

# Instalación de librerías (descomentar si es necesario ejecutarlo)
# !pip install transformers torch pandas

# Importación de las librerías necesarias para el proyecto
from transformers import pipeline  # Para crear el pipeline de inferencia de Hugging Face.
import pandas as pd              # Para la manipulación de datos tabulares (DataFrames).
import torch                     # El backend de deep learning.
from collections import Counter      # Utilidad para contar elementos (ej. distribución de estrellas).

print("✓ Librerías cargadas correctamente.")
print(f"✓ Versión de PyTorch: {torch.__version__}")
# Comprobamos si PyTorch puede acceder a una GPU compatible con CUDA.
# Esto es crucial para optimizar el rendimiento.
print(f"✓ GPU disponible: {'Sí' if torch.cuda.is_available() else 'No (usando CPU)'}\n")

✓ Librerías cargadas correctamente.
✓ Versión de PyTorch: 2.8.0+cu126
✓ GPU disponible: No (usando CPU)



-----

## **Sección 2: Creación y Estructuración del Dataset**

El corazón de cualquier proyecto de Machine Learning son los datos. Para este ejercicio, simularemos un conjunto de datos representativo.

1.  **Diseño del Dataset**: Creamos una lista de 10 reseñas clínicas en español. La selección no es aleatoria; está diseñada para cubrir un espectro de sentimientos: **muy positivos**, **muy negativos**, **neutrales** y **mixtos/ambiguos**. Esta variedad es crucial para evaluar la capacidad del modelo para captar matices, como en la reseña `"Las instalaciones son modernas y limpias, pero la espera fue de más de 2 horas"`, que contiene elementos positivos y negativos.
2.  **Estructuración con Pandas**: Los datos brutos (una lista de texto) se estructuran en un **DataFrame de Pandas**. Esta estructura tabular es el estándar de facto en ciencia de datos, ya que facilita enormemente la manipulación, el análisis y la adición de nuevas columnas, como haremos más adelante con las predicciones del modelo.
3.  **Mejora de la Legibilidad**: Se ajusta el índice del DataFrame para que comience en `1` en lugar del `0` predeterminado. Este es un pequeño cambio estético que mejora significativamente la legibilidad al presentar los resultados, haciéndolos más intuitivos para un público no técnico.
4.  **Metadatos del Dataset**: Finalmente, se imprimen metadatos básicos como el número total de reseñas y la longitud promedio de los textos. Esto nos da una primera idea del tamaño y la complejidad de nuestro corpus.

<!-- end list -->

In [ ]:
# ===================================================================
# SECCIÓN 2: CREACIÓN DEL DATASET SIMULADO
# ===================================================================

# Dataset con 10 reseñas clínicas diseñadas para cubrir un rango de sentimientos.
reseñas_clinicas = [
    "La atención fue excelente desde que entré. El doctor se tomó el tiempo para explicar todo detalladamente y respondió todas mis preguntas.",
    "El tratamiento ha sido increíblemente efectivo. Me siento mucho mejor y los síntomas han desaparecido por completo.",
    "Las instalaciones son modernas y limpias, pero la espera fue de más de 2 horas. Deberían mejorar la gestión de turnos.",
    "No resolvieron mi problema. Salí con las mismas dudas con las que llegué y nadie me dio seguimiento.",
    "El personal de recepción fue bastante grosero y poco servicial. Tuve que insistir varias veces para que me atendieran.",
    "Es un centro médico aceptable, ni bueno ni malo. Cumple con lo básico pero nada destacable.",
    "Me sentí completamente ignorado por el personal médico. La doctora apenas me miró, no volvería jamás.",
    "La doctora fue muy profesional y empática. Se nota su vocación y experiencia. Una experiencia muy positiva.",
    "El procedimiento fue rápido, aunque un poco doloroso. Los resultados parecen buenos según el especialista.",
    "Recibí el diagnóstico correcto después de visitar a varios especialistas sin respuesta. Muy agradecido con el equipo médico."
]

# Creamos un DataFrame, el estándar para trabajar con datos estructurados en Python.
df = pd.DataFrame(reseñas_clinicas, columns=['reseña'])
df.index = df.index + 1  # Modificamos el índice para que inicie en 1, facilitando la referencia.

print("=" * 70)
print("DATASET SIMULADO CREADO")
print("=" * 70)
print(f"Total de reseñas: {len(df)}")
print(f"Longitud promedio: {df['reseña'].str.len().mean():.0f} caracteres\n")

DATASET SIMULADO CREADO
Total de reseñas: 10
Longitud promedio: 112 caracteres



-----

## **Sección 3: Carga del Modelo Transformer**

Este es el paso más importante. Aquí cargamos el "cerebro" de nuestro sistema: un modelo Transformer pre-entrenado.

1.  **Selección del Modelo**: Elegimos `nlptown/bert-base-multilingual-uncased-sentiment`. Esta elección es estratégica:
      * **BERT (Bidirectional Encoder Representations from Transformers)**: Es una arquitectura que, a diferencia de modelos anteriores, lee el texto completo de una vez (de ahí "bidireccional"), lo que le permite entender el contexto de una palabra basándose en *todas* las demás palabras de la oración.
      * **Multilingual**: Pre-entrenado en un corpus de más de 100 idiomas, aprende representaciones que son sorprendentemente efectivas para muchos idiomas a la vez, incluido el español, sin necesidad de traducción.
      * **Sentiment-tuned**: Después de su pre-entrenamiento general, este modelo fue "afinado" (fine-tuned) en una tarea específica: clasificar textos en una escala de 1 a 5 estrellas. Este es un ejemplo de **Transfer Learning**, la técnica más poderosa del NLP moderno.
2.  **Instanciación del `pipeline`**: La función `pipeline` de Hugging Face es una abstracción de alto nivel que maneja toda la complejidad. Al llamarla, automáticamente:
      * Descarga los pesos del modelo y su configuración desde el Hub de Hugging Face.
      * Carga el tokenizador específico del modelo, que convierte el texto en un formato numérico que BERT puede entender (usando la técnica de sub-palabras WordPiece).
      * Configura el modelo para inferencia.
3.  **Asignación de Hardware (Device)**: Aquí utilizamos el resultado de nuestra comprobación anterior. El argumento `device=0` le dice a PyTorch que mueva el modelo y los datos a la primera GPU disponible. Si no hay GPU (`is_available()` es `False`), `device=-1` le indica que use la CPU. Esto asegura que el código sea portable y siempre utilice el hardware más rápido disponible.
4.  **Manejo de Errores**: El bloque `try...except` asegura que si la descarga o carga del modelo falla (un problema común con conexiones de red inestables), el script no se detendrá abruptamente, sino que informará del problema de manera controlada.

<!-- end list -->

In [ ]:
# ===================================================================
# SECCIÓN 3: CARGA DEL MODELO PRE-ENTRENADO
# ===================================================================

# Especificamos el identificador del modelo en el Hub de Hugging Face.
nombre_modelo = "nlptown/bert-base-multilingual-uncased-sentiment"

print("=" * 70)
print("CARGANDO MODELO TRANSFORMER")
print("=" * 70)
print(f"Modelo: {nombre_modelo}")
print("Características:")
print("  • Arquitectura: BERT (Bidirectional Encoder Representations)")
print("  • Idiomas: 100+ incluyendo español")
print("  • Tarea: Análisis de sentimientos (1-5 estrellas)")
print("\nCargando... (esto puede tardar la primera vez mientras se descarga el modelo)")

try:
    # Creamos el pipeline, que gestiona tokenización, inferencia y post-procesado.
    clasificador_sentimientos = pipeline(
        "sentiment-analysis",
        model=nombre_modelo,
        device=0 if torch.cuda.is_available() else -1  # Asignación dinámica a GPU o CPU.
    )
    print("✓ Modelo cargado exitosamente\n")
except Exception as e:
    print(f"⚠ Error al cargar el modelo: {e}")
    print("Verifica tu conexión a internet y que el nombre del modelo sea correcto.\n")
    raise

CARGANDO MODELO TRANSFORMER
Modelo: nlptown/bert-base-multilingual-uncased-sentiment
Características:
  • Arquitectura: BERT (Bidirectional Encoder Representations)
  • Idiomas: 100+ incluyendo español
  • Tarea: Análisis de sentimientos (1-5 estrellas)

Cargando... (esto puede tardar la primera vez mientras se descarga el modelo)


Device set to use cpu


✓ Modelo cargado exitosamente



-----

## **Sección 4: Clasificación y Enriquecimiento de Datos**

Con el modelo cargado y el dataset listo, procedemos a la inferencia y a la integración de los resultados.

1.  **Inferencia en Lote (Batch Processing)**: En lugar de iterar y predecir una por una, pasamos la lista completa de reseñas (`reseñas_clinicas`) al `clasificador`. La librería `transformers` está optimizada para procesar datos en lotes, lo que es mucho más eficiente, especialmente en una GPU.
2.  **Extracción de Resultados**: El pipeline devuelve una lista de diccionarios, cada uno con una `label` (ej. `'5 stars'`) y un `score` (la confianza del modelo en esa predicción, de 0 a 1). Usamos "list comprehension" (`[... for ... in ...]`) para extraer de manera eficiente y concisa estos valores en dos listas separadas: `predicciones` y `scores`.
3.  **Ingeniería de Características (Feature Engineering)**: No nos limitamos a guardar los resultados brutos. Creamos nuevas columnas que aportan más valor:
      * `confianza`: Se formatea el `score` como un porcentaje para que sea más fácil de interpretar.
      * `estrellas`: Se extrae solo el número de la `label` (ej. `'5'` de `'5 stars'`) para facilitar análisis numéricos posteriores.
      * `interpretacion`: Se aplica una función `interpretar_sentimiento` para mapear la puntuación numérica a una categoría semántica con un emoji. Esto es clave para la visualización y la comunicación de resultados a un público no técnico.
4.  **Presentación Detallada**: Finalmente, en lugar de un simple `print(df)`, se itera sobre el DataFrame enriquecido para presentar un "informe" por cada reseña. Este formato es ideal para un análisis cualitativo, ya que permite ver el texto original y su clasificación completa de un solo vistazo.

<!-- end list -->

In [ ]:
# ===================================================================
# SECCIÓN 4: CLASIFICACIÓN Y VISUALIZACIÓN DE RESULTADOS
# ===================================================================

print("=" * 70)
print("CLASIFICANDO RESEÑAS (INFERENCIA)")
print("=" * 70)

# El pipeline puede procesar una lista de textos de manera eficiente.
resultados = clasificador_sentimientos(reseñas_clinicas)

# Extraemos las etiquetas y los puntajes de confianza de la salida del modelo.
predicciones = [resultado['label'] for resultado in resultados]
scores = [resultado['score'] for resultado in resultados]

# Agregamos los resultados al DataFrame, creando nuevas columnas.
df['prediccion'] = predicciones
# Formateamos el score como un porcentaje legible.
df['confianza'] = [f"{score*100:.1f}%" for score in scores]
# Extraemos solo el número de estrellas para análisis cuantitativo.
df['estrellas'] = [pred.split()[0] for pred in predicciones]

# Función auxiliar para mapear la predicción numérica a una etiqueta cualitativa.
def interpretar_sentimiento(estrellas):
    interpretacion = {
        '1': '😞 Muy Negativo',
        '2': '😕 Negativo',
        '3': '😐 Neutral',
        '4': '🙂 Positivo',
        '5': '😊 Muy Positivo'
    }
    return interpretacion.get(estrellas, 'Desconocido')

# Aplicamos la función para crear una columna final de interpretación.
df['interpretacion'] = df['estrellas'].apply(interpretar_sentimiento)

# Mostramos los resultados detallados en un formato claro y estructurado.
print("\nRESULTADOS DETALLADOS:\n")
for idx, row in df.iterrows():
    print(f"{'─' * 70}")
    print(f"Reseña #{idx}")
    print(f"{'─' * 70}")
    print(f"Texto: \"{row['reseña']}\"")
    print(f"\n→ Clasificación: {row['interpretacion']}")
    print(f"→ Estrellas: {row['estrellas']}/5")
    print(f"→ Confianza del modelo: {row['confianza']}")
    print()

CLASIFICANDO RESEÑAS (INFERENCIA)

RESULTADOS DETALLADOS:

──────────────────────────────────────────────────────────────────────
Reseña #1
──────────────────────────────────────────────────────────────────────
Texto: "La atención fue excelente desde que entré. El doctor se tomó el tiempo para explicar todo detalladamente y respondió todas mis preguntas."

→ Clasificación: 😊 Muy Positivo
→ Estrellas: 5/5
→ Confianza del modelo: 65.8%

──────────────────────────────────────────────────────────────────────
Reseña #2
──────────────────────────────────────────────────────────────────────
Texto: "El tratamiento ha sido increíblemente efectivo. Me siento mucho mejor y los síntomas han desaparecido por completo."

→ Clasificación: 😊 Muy Positivo
→ Estrellas: 5/5
→ Confianza del modelo: 64.6%

──────────────────────────────────────────────────────────────────────
Reseña #3
──────────────────────────────────────────────────────────────────────
Texto: "Las instalaciones son modernas y limpias, p

### **Interpretación Inmediata de Resultados del Modelo de Sentimiento**

Este análisis evalúa el rendimiento del modelo `BERT multilingüe` en la tarea de clasificación de sentimientos para un conjunto de 10 reseñas clínicas. La interpretación se centra no solo en la precisión de la etiqueta asignada, sino también en el significado y las implicaciones de los *scores* de confianza.

***

### 📊 **1. Resumen Ejecutivo**

El modelo demuestra un **rendimiento general sobresaliente**, clasificando correctamente el sentimiento de las 10 reseñas de acuerdo con un criterio humano. Es particularmente efectivo en **distinguir matices** entre sentimientos muy negativos, negativos, neutrales y positivos. El análisis de los niveles de confianza revela que el modelo es más "seguro" de sus predicciones cuando el lenguaje es inequívoco y fuertemente emocional, mientras que su confianza disminuye ante textos con **sentimientos mixtos** o **vocabulario muy especializado**, lo cual es un comportamiento esperado y útil.

***

### 🎯 **2. Análisis de Rendimiento y Capacidades del Modelo**

#### **Alta Precisión en la Clasificación**
El hallazgo principal es que el modelo asignó una etiqueta de sentimiento (`interpretacion` y `estrellas`) que se alinea perfectamente con la intención de cada reseña.
* **Ejemplos Positivos**: Reseñas como la #1 ("excelente atención") y la #8 ("doctora muy profesional") fueron correctamente identificadas como **"Muy Positivo" (5/5)**.
* **Ejemplos Negativos**: Reseñas como la #4 ("No resolvieron mi problema") y la #7 ("Me sentí completamente ignorado") recibieron la calificación más baja (**1/5**), lo cual es apropiado.

#### **Excelente Manejo de la Ambigüedad y Neutralidad**
La verdadera capacidad de un modelo de lenguaje se observa en los casos no triviales.
* **Sentimientos Mixtos**: La **Reseña #3** ("instalaciones modernas y limpias, *pero* la espera fue de más de 2 horas") es el mejor ejemplo. El modelo ponderó correctamente el elemento positivo y el negativo, asignando una calificación **"Neutral" (3/5)**. Esto demuestra que no es un simple contador de palabras clave, sino que entiende la estructura de la oración.
* **Neutralidad Explícita**: La **Reseña #6** ("aceptable, ni bueno ni malo") fue clasificada como **"Neutral" (3/5)** con una confianza relativamente alta (69.5%), mostrando que puede identificar la ausencia de un sentimiento fuerte.

#### **Distinción Fina en la Escala de Sentimiento**
Es notable la diferencia entre la **Reseña #5** y la **#7**.
* La #5 ("personal grosero") fue clasificada como **"Negativo" (2/5)**.
* La #7 ("completamente ignorado... no volvería jamás") fue clasificada como **"Muy Negativo" (1/5)**.
Esta distinción es coherente: mientras que un trato "grosero" es una mala experiencia (2 estrellas), el sentimiento de ser "ignorado" y la decisión de "no volver jamás" denotan un fallo total en el servicio (1 estrella).

***

### 💡 **3. Análisis Profundo de los Niveles de Confianza**

El *score* de confianza es una métrica crucial que nos dice qué tan "seguro" está el modelo de su propia predicción.

#### **La Confianza como Indicador de Claridad del Texto**
Hay una correlación directa entre la claridad del sentimiento en el texto y la confianza del modelo.
* **Confianza Máxima (80.1%)**: Se observa en la **Reseña #7**. El lenguaje es directo, fuertemente emocional y no deja lugar a dudas ("completamente ignorado", "jamás"). Para el modelo, esta es una señal inequívoca.
* **Confianza Mínima (43.3%)**: Ocurre en la **Reseña #3**, la de sentimientos mixtos. La baja confianza **no es un error**, sino una señal valiosa. El modelo nos está diciendo: "Clasifico esto como neutral, pero reconozco que hay fuertes señales contradictorias en el texto".

#### **El Impacto del Vocabulario Específico del Dominio**
Un hallazgo muy interesante es la confianza en la **Reseña #10** (53.8%).
* Aunque el sentimiento es claramente muy positivo ("diagnóstico correcto", "muy agradecido"), la confianza es la más baja entre todas las reseñas positivas.
* **¿Por qué?** Probablemente porque el modelo base fue entrenado con textos de internet generales (reseñas de productos, películas, etc.), donde palabras como "diagnóstico", "especialistas" o "equipo médico" son menos comunes en contextos de 5 estrellas. El modelo clasifica correctamente el sentimiento general, pero la terminología "atípica" reduce su certeza.

***

### ✅ **4. Conclusiones y Recomendaciones Prácticas**

1.  **Validación del Modelo**: El modelo `nlptown/bert-base-multilingual-uncased-sentiment` es **altamente efectivo y fiable** para una primera capa de clasificación automatizada de reseñas clínicas en español.

2.  **Utilizar la Confianza como Filtro**: La confianza es más que un simple número; es una herramienta de priorización. Se recomienda establecer una regla de negocio:
    * **Acción Sugerida**: Todas las reseñas con una **confianza inferior al 50% o 60%** deberían ser marcadas automáticamente para una **revisión humana prioritaria**. Estos son los casos más ambiguos o complejos donde un análisis humano puede aportar el mayor valor.

3.  **Oportunidad de Mejora (Fine-Tuning)**: La menor confianza en reseñas con terminología médica (#10) revela la principal limitación de usar un modelo de propósito general.
    * **Próximo Paso Estratégico**: Para llevar este sistema a un nivel de producción robusto, el siguiente paso sería recolectar un dataset de varios cientos de reseñas clínicas propias y realizar un **"fine-tuning" (re-entrenamiento fino)** del modelo. Esto lo especializaría en el lenguaje específico del dominio de la salud, aumentando su precisión y, sobre todo, su confianza en casos relevantes.

-----

## **Sección 5: Análisis Cuantitativo Agregado**

Tras el análisis individual, es crucial obtener una visión panorámica del conjunto de datos. Este bloque realiza un análisis agregado para resumir los hallazgos.

1.  **Distribución de Sentimientos**: Usamos `collections.Counter` para contar la frecuencia de cada calificación en estrellas. Esto nos da una rápida visión de la polaridad general de las reseñas. La visualización con barras de texto (`'█'`) es una forma sencilla pero efectiva de crear un histograma directamente en la consola.
2.  **Cálculo de Métricas Clave**:
      * **Conversión de Tipo**: La columna `estrellas` (que es texto) se convierte a tipo numérico (`int`) para poder realizar operaciones matemáticas.
      * **Métricas de Resumen**: Se calculan estadísticas agregadas como la **puntuación promedio** y la **confianza promedio del modelo**. Estas métricas son útiles para monitorizar el sentimiento general a lo largo del tiempo.
      * **Segmentación**: Se cuenta el número de reseñas que caen en categorías predefinidas (positivas, neutrales, negativas). Esta segmentación es fundamental en un contexto de negocio para, por ejemplo, cuantificar el porcentaje de clientes satisfechos o insatisfechos.

<!-- end list -->

In [ ]:
# ===================================================================
# SECCIÓN 5: ANÁLISIS CUANTITATIVO
# ===================================================================

print("=" * 70)
print("ANÁLISIS CUANTITATIVO AGREGADO")
print("=" * 70)

# Usamos Counter para obtener la frecuencia de cada calificación.
distribucion = Counter(df['estrellas'])
print("\nDistribución de clasificaciones:")
for estrella in sorted(distribucion.keys()):
    cantidad = distribucion[estrella]
    barra = '█' * cantidad
    print(f"  {estrella} estrella(s): {barra} ({cantidad})")

# Convertimos la columna 'estrellas' a numérica para poder calcular estadísticas.
estrellas_numericas = df['estrellas'].astype(int)
promedio = estrellas_numericas.mean()
# Para la confianza, volvemos a usar la lista original de 'scores' (números, no texto).
confianza_promedio = pd.Series(scores).mean() * 100

print(f"\n📊 Estadísticas Generales:")
print(f"  • Puntuación promedio: {promedio:.2f}/5.0")
print(f"  • Confianza promedio del modelo: {confianza_promedio:.1f}%")
print(f"  • Reseñas positivas (4-5★): {len(df[estrellas_numericas >= 4])}")
print(f"  • Reseñas neutrales (3★): {len(df[estrellas_numericas == 3])}")
print(f"  • Reseñas negativas (1-2★): {len(df[estrellas_numericas <= 2])}")

ANÁLISIS CUANTITATIVO AGREGADO

Distribución de clasificaciones:
  1 estrella(s): ██ (2)
  2 estrella(s): █ (1)
  3 estrella(s): ███ (3)
  5 estrella(s): ████ (4)

📊 Estadísticas Generales:
  • Puntuación promedio: 3.30/5.0
  • Confianza promedio del modelo: 63.1%
  • Reseñas positivas (4-5★): 4
  • Reseñas neutrales (3★): 3
  • Reseñas negativas (1-2★): 3


### **Interpretación del Análisis Cuantitativo Agregado**

Este informe presenta una evaluación de los datos agregados obtenidos del modelo de clasificación de sentimientos. El análisis va más allá de los números superficiales para descubrir patrones en la opinión de los pacientes y proponer acciones estratégicas.

***

### 📊 **1. Resumen Ejecutivo**

Los resultados cuantitativos revelan una **fuerte polarización en la experiencia de los pacientes**. Lejos de ser una institución mediocre, el centro genera opiniones extremas: un grupo significativo de pacientes muy satisfechos y un grupo igualmente considerable de pacientes insatisfechos. El puntaje promedio de **3.30 sobre 5.0** es estadísticamente engañoso si se analiza de forma aislada, ya que enmascara esta división en la percepción del servicio. La confianza promedio del modelo, de **63.1%**, es saludable e indica un rendimiento fiable en un corpus con lenguaje variado.

***

### 📈 **2. Análisis de la Distribución de Sentimientos**

La distribución de las calificaciones es el hallazgo más importante de este análisis.

#### **Polarización en Vez de Centralización**
La distribución `1-1-3-0-4` (para 1, 2, 3, 4 y 5 estrellas respectivamente) es una curva en forma de "U". Esto significa que las experiencias no se agrupan en el centro (calificaciones de 3 estrellas), sino en los extremos.
* **Promotores Fuertes (40%)**: Existe un sólido grupo de 4 reseñas de **5 estrellas**. Esto indica que, cuando el servicio funciona bien, la experiencia es excelente y genera una alta satisfacción.
* **Detractores Significativos (30%)**: Hay 3 reseñas en el espectro negativo (**1 y 2 estrellas**). Esto demuestra que existen fallos operativos o de servicio que generan una insatisfacción considerable.

#### **La Ausencia Clave: Las 4 Estrellas**
Es sumamente revelador que **no haya ninguna reseña de 4 estrellas**. En un contexto de servicio al cliente, esto sugiere que las experiencias tienden a ser binarias: o son **excelentes (5 estrellas)** o tienen algún fallo notable que las degrada a **neutrales (3 estrellas)** o peores. No parece haber un término medio de "bueno, pero no excelente".

***

### 🧐 **3. Interpretación de las Estadísticas Generales**

#### **Puntuación Promedio (3.30/5.0): Una Métrica Engañosa**
Un promedio de 3.30 podría llevar a la conclusión errónea de que la mayoría de los pacientes tienen una opinión "ligeramente positiva". Sin embargo, la distribución nos muestra que este promedio no es el resultado de muchas calificaciones de 3 y 4, sino el **promedio matemático de extremos opuestos**. Ignorar la distribución y centrarse solo en el promedio ocultaría los problemas críticos y las fortalezas clave de la institución.

#### **Confianza Promedio del Modelo (63.1%)**
Una confianza del 63.1% es un indicador positivo de la fiabilidad del modelo. Significa que, en promedio, el clasificador tuvo una certeza considerable en sus predicciones. No es una confianza extremadamente alta (ej. >90%), lo que es normal y esperado, ya que el lenguaje humano contiene ambigüedades y matices que el modelo capta, reflejando esta complejidad en sus scores de confianza.

***

#### **4. Conclusiones y Acciones Recomendadas**

Los datos cuantitativos no son solo para observar, sino para actuar. A partir de este análisis, se proponen las siguientes acciones:

1.  **Investigar los Polos de Experiencia**:
    * **Acción**: Realizar un análisis cualitativo profundo de las **4 reseñas de 5 estrellas** para identificar qué procesos, actitudes o servicios generan esa "experiencia perfecta". Estos son los puntos fuertes que se deben estandarizar y replicar.
    * **Acción**: Tratar las **3 reseñas negativas (1 y 2 estrellas)** como alertas críticas. Analizar sus causas raíz (ej. tiempos de espera, trato del personal, errores de procedimiento) para implementar acciones correctivas inmediatas.

2.  **Enfocarse en el Segmento Neutral**:
    * **Acción**: Las **3 reseñas de 3 estrellas** representan la mayor oportunidad de mejora. Estos pacientes no tuvieron una mala experiencia, pero tampoco una memorable. Analizar sus comentarios es clave para entender qué pequeños cambios podrían elevar su experiencia de "aceptable" a "excelente" y convertirlos en promotores.

3.  **Establecer un Sistema de Monitoreo**:
    * **Acción**: Implementar este análisis como un **dashboard de seguimiento continuo**. El objetivo no es solo mantener un promedio de 3.30, sino **cambiar la forma de la distribución**: reducir el número de reseñas de 1 y 2 estrellas y aumentar las de 4 y 5 a lo largo del tiempo, demostrando el impacto de las mejoras operativas.

-----

## **Sección 6: Análisis Final de Rendimiento del Modelo Transformer**

Tras la ejecución del pipeline de clasificación sobre el dataset de 10 reseñas clínicas, procedemos a realizar un análisis crítico y exhaustivo. Este análisis responde a las preguntas clave sobre la correspondencia de los resultados con el juicio humano, las limitaciones inherentes del modelo y sus ventajas fundamentales sobre enfoques tradicionales.

***

### **1. ¿Los resultados coinciden con tu criterio humano?**

**Sí, la coincidencia es excepcionalmente alta y robusta.** El modelo no solo asignó la etiqueta de sentimiento correcta a cada una de las 10 reseñas, sino que los *scores* de confianza que acompañan a cada predicción actúan como un reflejo coherente de la complejidad y ambigüedad del texto, alineándose perfectamente con la intuición humana.

* ✅ **Precisión en Sentimientos Extremos y Claros**:
    El modelo identificó sin problemas las reseñas con una carga emocional inequívoca. La **reseña #7** ("*Me sentí completamente ignorado por el personal médico... no volvería jamás*") fue clasificada como **1 estrella** con una **confianza del 80.1%**, la más alta de todo el conjunto de datos. Esto demuestra que el lenguaje directo y fuertemente negativo genera una señal clara que el modelo capta con gran seguridad. De igual manera, reseñas positivas como la **#8** ("*La doctora fue muy profesional y empática*") fueron correctamente clasificadas con **5 estrellas** y una confianza sólida (67.7%).

* ✅ **Captura Inteligente de la Ambigüedad**:
    El verdadero test de inteligencia de un modelo de NLP reside en su manejo de textos con sentimientos mixtos. En este aspecto, el rendimiento fue sobresaliente.
    La **reseña #3** ("*Las instalaciones son modernas y limpias, pero la espera fue de más de 2 horas*") recibió una calificación de **3 estrellas (Neutral)**. Esta es la interpretación humana perfecta. Aún más revelador es que esta predicción tuvo la **confianza más baja de todas (43.3%)**. Esto **no es un fallo**, sino una manifestación de la sofisticación del modelo: su baja confianza es una señal de que reconoce las fuerzas semánticas opuestas ("modernas y limpias" vs. "espera larga") dentro del mismo texto.

* ✅ **Correcta Interpretación de la Neutralidad**:
    La **reseña #6** ("*Es un centro médico aceptable, ni bueno ni malo*") fue clasificada como **3 estrellas** con una confianza relativamente alta (69.5%). Esto demuestra que el modelo no solo reacciona a palabras clave de sentimiento, sino que también entiende las frases que denotan explícitamente una experiencia neutral o promedio.

***

### **2. ¿Qué limitaciones observas del modelo?**

A pesar de su excelente rendimiento en esta tarea, el análisis de los resultados y la naturaleza del modelo nos permiten identificar varias limitaciones cruciales que deben considerarse antes de una implementación en un entorno real.

* **Falta de Contexto Médico Específico**:
    Esta limitación teórica se ve confirmada por los resultados. La **reseña #10** ("*Recibí el diagnóstico correcto... Muy agradecido*") fue clasificada correctamente con **5 estrellas**, pero con una **confianza sorprendentemente baja de 53.8%**. La explicación más probable es que el modelo, entrenado en textos generales de internet, no está familiarizado con el vocabulario del dominio de la salud ("diagnóstico", "especialistas"). Aunque infiere el sentimiento positivo del contexto, la terminología "extraña" reduce su certeza. Esto evidencia el riesgo de que el modelo malinterprete frases con significado clínico específico, como **"el resultado fue negativo"** (una buena noticia en medicina).

* **Incapacidad para Detectar Sarcasmo e Ironía**:
    Aunque nuestro dataset no incluía ejemplos de sarcasmo, esta sigue siendo una de las debilidades más conocidas de los modelos de lenguaje. Una reseña como "*Qué maravilla esperar tres horas para una consulta de cinco minutos*" sería, con alta probabilidad, clasificada erróneamente como positiva debido a la palabra "maravilla", ya que el modelo carece del razonamiento de sentido común para detectar la intención irónica.

* **Salida Rígida y Unidimensional**:
    El modelo condensa toda la información de una reseña en un único número (1-5). Esto es útil para una visión general, pero oculta detalles operativos vitales. La **reseña #3 (3 estrellas)** es un claro ejemplo: la calificación "neutral" no nos dice que el problema específico fue el **tiempo de espera**, mientras que las **instalaciones** fueron un punto positivo. Para la mejora de procesos, es fundamental extraer no solo el sentimiento general, sino también los temas o aspectos específicos a los que se refiere. El modelo actual nos da el "qué", pero no el "porqué".

***

### **3. ¿Qué ventajas tiene respecto a un modelo tradicional (TF-IDF)?**

Los modelos Transformer como BERT no son una mejora incremental; representan un salto paradigmático en la capacidad de comprender el lenguaje. Las ventajas sobre los métodos tradicionales como TF-IDF son fundamentales.

* 🚀 **Comprensión del Contexto vs. "Bolsa de Palabras"**:
    * **TF-IDF** trata el texto como un conjunto desordenado de palabras. Solo mide la frecuencia e importancia de cada palabra de forma aislada. Para este método, las frases "*El doctor NO fue amable*" y "*El doctor fue amable*" son casi idénticas, pues comparten la mayoría de las palabras.
    * **BERT**, gracias a su arquitectura de **atención bidireccional**, analiza cada palabra en relación con todas las demás en la oración. Entiende que la palabra "NO" invierte completamente el significado de "amable". Es esta capacidad contextual la que le permite descifrar la ambigüedad en la reseña #3, donde la palabra "pero" actúa como un punto de inflexión en el sentimiento.

* 🚀 **Transfer Learning y Eficiencia de Datos**:
    * **TF-IDF** necesita ser construido desde cero para cada problema. Requeriría un gran dataset (miles de reseñas etiquetadas manualmente) para aprender qué palabras se asocian con sentimientos positivos o negativos.
    * **BERT** viene **pre-entrenado** en un corpus masivo de texto (libros, Wikipedia, etc.). Ya posee un conocimiento profundo de la gramática, la sintaxis y las relaciones semánticas. La razón por la que obtuvimos resultados tan precisos con **solo 10 reseñas** y sin ningún entrenamiento adicional es gracias a este conocimiento previo. Esto democratiza el acceso a tecnología de NLP avanzada para problemas con datos limitados.

* 🚀 **Manejo de Vocabulario y Semántica Profunda (Subword Tokenization)**:
    * **TF-IDF** tiene un vocabulario fijo. Si se encuentra con una palabra que no vio en el entrenamiento (un término médico como "electrocardiograma" o un simple error de tipeo), la ignora, perdiendo información valiosa.
    * **BERT** utiliza **tokenización de sub-palabras**. Puede descomponer palabras desconocidas en partes más pequeñas que sí conoce (ej. "electrocardiograma" → "electro" + "cardio" + "grama"). Esto le da una flexibilidad casi infinita para manejar cualquier texto. Además, su entrenamiento le permite entender que palabras como "excelente", "profesional" y "efectivo", aunque distintas, contribuyen a un sentimiento semánticamente similar (muy positivo), algo que TF-IDF solo podría aprender por correlación estadística tras ver miles de ejemplos.